In [1]:
class Sudoku():
    """A sudoku puzzle"""
    rows = 'ABCDEFGHI'
    cols = '123456789'

    def __init__(self, initial_grid, game_type='standard'):
        """ Initialize the Sudoku puzzle.
            Args:
                - initial_grid (str): string representing the sudoku grid
                  Ex. '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
                - game_type (str): the type of game to define rules
                    Choices:
                        1. 'standard': use the normal rules of 1-9 appearing exactly once in each row, col, and square
                        2. 'standard+diagonal': use the 'standard' rules above and also require 1-9 to appear exactly once in each diagonal
        """
        if len(initial_grid) != 81:
            raise Exception("Grid must be 81 characters")
        self.build_puzzle(game_type)
        self.game_type = game_type
        self.initial_grid = initial_grid
        self.grid = initial_grid
        self.values = self.grid_to_values(initial_grid)

    def build_puzzle(self, game_type):
        """ Build helpful attributes about the sudoku puzzle for a given type of rules.
            Args:
                - game_type (str): the type of game to define rules
        """
        self.boxes = [r + c for r in self.rows for c in self.cols]
        self.row_units = [self.cross(r, self.cols) for r in self.rows]
        self.column_units = [self.cross(self.rows, c) for c in self.cols]
        self.square_units = [self.cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
        self.diagonal_units = []
        self.unitlist = self.row_units + self.column_units + self.square_units

        if game_type == 'standard+diagonal':
            self.diagonal_units = [
                [self.rows[position] + self.cols[position] for position in range(0, len(self.rows))],
                [self.rows[position] + self.cols[-(position + 1)] for position in range(0, len(self.rows))]
            ]
            self.unitlist += self.diagonal_units

        self.units = dict((s, [u for u in self.unitlist if s in u]) for s in self.boxes)
        self.peers = dict((s, set(sum(self.units[s],[]))-set([s])) for s in self.boxes)

    @staticmethod
    def cross(A, B):
        """Cross product of elements in A and elements in B """
        return [x+y for x in A for y in B]

    def values_to_grid(self, values=None):
        """ Convert the dictionary board representation to as string.
            Args:
                - values (dict): boxes map to their values strings (i.e. 'A1' => '135')
            Returns:
                - grid (str): string representing the sudoku grid
                  Ex. '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
        """
        if values is None:
            values = self.values

        res = []
        for r in self.rows:
            for c in self.cols:
                v = values[r + c]
                res.append(v if len(v) == 1 else '.')
        return ''.join(res)

    def grid_to_values(self, grid=None):
        """ Convert grid into a dict of {square: char} with '123456789' for empties.
            Args:
                - grid (str): string representing the sudoku grid
                  Ex. '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
            Returns:
                - values (dict): boxes map to their values strings (i.e. 'A1' => '135')
                                 Empty values will be set to '123456789'
        """
        if grid is None:
            grid = self.grid

        sudoku_grid = {}
        for val, key in zip(grid, self.boxes):
            if val == '.':
                sudoku_grid[key] = '123456789'
            else:
                sudoku_grid[key] = val
        return sudoku_grid

    def remove_digit(self, box, digit):
        """ Remove a digit from the possible values of a box.
            Args:
                - box (str): the box to remove the digit from
                - digit (str): the digit to remove from the box
            Returns:
                - values (dict): updated values dict with the digit removed from the box
        """
        self.values[box] = self.values[box].replace(digit, '')

    def display(self, values=None):
        """ Display the values as a 2-D grid.
            Args:
                - values (dict): boxes map to their values strings (i.e. 'A1' => '135')
        """
        if values is None:
            values = self.values

        width = 1+max(len(values[s]) for s in self.boxes)
        line = '+'.join(['-'*(width*3)]*3)
        for r in self.rows:
            print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                          for c in self.cols))
            if r in 'CF': print(line)
        print()

    def is_solved(self):
        """ Determine if the puzzle has been solved.
            Returns:
                - solved (boolean): whether or not the puzzle is solved
        """
        fully_reduced = all(len(self.values[s]) == 1 for s in self.boxes)
        if not fully_reduced:
            return False

        for unit in self.unitlist:
            required_digits = '123456789'
            for box in unit:
                required_digits = required_digits.replace(self.values[box], '')
            if len(required_digits) != 0:
                return False
        return True

In [2]:
class SudokuSolver():
    """Class that solves sudoku puzzles"""
    rows = 'ABCDEFGHI'
    cols = '123456789'

    def configure_solver(self, puzzle):
        """ Build helpful attributes about the sudoku puzzle for a given type of rules.
            Args:
                - puzzle (obj): the instance of a Sudoku we are trying to solve
        """
        self.boxes = puzzle.boxes
        self.unitlist = puzzle.unitlist
        self.units = puzzle.units
        self.peers = puzzle.peers

    def naked_twins(self, values):
        """ Eliminate values using the naked twins strategy.
            Args:
                - values (dict): boxes map to their values strings (i.e. 'A1' => '135')
            Returns:
                - values (dict): the values dict with naked twins eliminated
        """
        for unit in self.unitlist:
            twin_values = self.find_twins(values, unit)
            values = self.eliminate_twin_values(values, unit, twin_values)
        return values

    def eliminate_twin_values(self, values, unit, twin_values):
        """ Remove the twin values from a unit.
            Args:
                - values (dict): boxes map to their values strings (i.e. 'A1' => '135')
                - unit (list): list of boxes in a unit
                - twin_values (list): list of twin values to eliminate from unit
            Returns:
                - values (dict): updated values dict with twins eliminated from the unit
        """
        for box in unit:
            if values[box] in twin_values:
                continue
            for twin in twin_values:
                for digit in twin:
                    values = self.remove_digit(values, box, digit)
        return values

    def find_twins(self, values, unit):
        """ Find the current twins for a unit (row, column, etc.).
            Args:
                - values (dict): boxes map to their values strings (i.e. 'A1' => '135')
                - unit (list): list of boxes in a unit
            Returns:
                - twin_values (list): the values that are length two and occur exactly twice in the unit
        """
        unit_values = [values[box] for box in unit]
        return [value for value in unit_values if unit_values.count(value) == 2 and len(value) == 2]

    @staticmethod
    def remove_digit(values, box, digit):
        """ Remove a digit from the possible values of a box.
            Args:
                - values (dict): boxes map to their values strings (i.e. 'A1' => '135')
                - box (str): the box to remove the digit from
                - digit (str): the digit to remove from the box
            Returns:
                - values (dict): updated values dict with the digit removed from the box
        """
        values[box] = values[box].replace(digit, '')
        return values


    def eliminate(self, values):
        """ Apply the eliminate strategy to a Sudoku puzzle.
            The eliminate strategy says that if a box has a value assigned, then none
            of the peers of that box can have the same value.
            Args:
                - values (dict): boxes map to their values strings (i.e. 'A1' => '135')
            Returns:
                - values (dict): the values dict with solved values eliminated from peers
        """
        for box, value in values.items():
            if len(value) == 1:
                for peer in self.peers[box]:
                    values = self.remove_digit(values, peer, value)
        return values


    def only_choice(self, values):
        """ Apply the only choice strategy to a Sudoku puzzle.
            The only choice strategy says that if only one box in a unit allows a certain
            digit, then that box must be assigned that digit.
            Args:
                - values (dict): boxes map to their values strings (i.e. 'A1' => '135')
            Returns:
                - values (dict): the values dict with only choice values solved
        """
        for group in self.unitlist:
            for digit in '123456789':
                remainder = [box for box in group if digit in values[box]]
                if len(remainder) == 1:
                    values[remainder[0]] = digit
        return values


    def reduce_puzzle(self, values):
        """ Reduce a Sudoku puzzle by repeatedly applying all constraint strategies.
            Args:
                - values (dict): boxes map to their values strings (i.e. 'A1' => '135')
            Returns:
                - values (dict/False): The values dictionary after continued application of
                  the constraint strategies no longer produces any changes, or False if the
                  puzzle is unsolvable
        """
        stalled = False
        while not stalled:
            solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])

            values = self.eliminate(values)

            values = self.naked_twins(values)

            values = self.only_choice(values)

            # Check how many boxes have a determined value, to compare
            solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
            # If no new values were added, stop the loop.
            stalled = solved_values_before == solved_values_after
            # Sanity check, return False if there is a box with zero available values:
            if len([box for box in values.keys() if len(values[box]) == 0]):
                return False
        return values

    def search(self, values):
        """ Apply depth first search to solve Sudoku puzzles in order to solve puzzles
            that cannot be solved by repeated reduction alone.
            Args:
                - values (dict): boxes map to their values strings (i.e. 'A1' => '135')
            Returns:
                - values (dict/False): The values dictionary after continued application of
                  the constraint strategies no longer produces any changes, or False if the
                  puzzle is unsolvable
        """
        values = self.reduce_puzzle(values)
        if values is False:
            return False ## Failed earlier
        if all(len(values[s]) == 1 for s in self.boxes):
            return values ## Solved!
        # Choose one of the unfilled squares with the fewest possibilities
        n,s = min((len(values[s]), s) for s in self.boxes if len(values[s]) > 1)
        # Now use recurrence to solve each one of the resulting sudokus, and
        for value in values[s]:
            new_sudoku = values.copy()
            new_sudoku[s] = value
            attempt = self.search(new_sudoku)
            if attempt:
                return attempt

    def solve(self, puzzle, display_solution=True):
        """Find the solution to a Sudoku puzzle using search and constraint propagation.
            Args:
                - puzzle (obj): an instance of a Sudoku
                - display_solution (boolean): whether or not to print the solution to sdout
        """
        if not isinstance(puzzle, Sudoku):
            raise Exception("The puzzle needs to be an instance of Sudoku")

        self.configure_solver(puzzle)
        puzzle.values = self.search(puzzle.values)

        if puzzle.is_solved():
            message = "The puzzle was solved!"
        else:
            message = "The solver failed :("

        if display_solution:
            print(message)
            puzzle.display()

In [3]:
grid = '8..........36......7..9.2...5...7.......457.....1...3...1....68..85...1..9....4..'

In [4]:
puzzle = Sudoku(grid)

In [5]:
solver = SudokuSolver()
solver.solve(puzzle)

The puzzle was solved!
8 1 2 |7 5 3 |6 4 9 
9 4 3 |6 8 2 |1 7 5 
6 7 5 |4 9 1 |2 8 3 
------+------+------
1 5 4 |2 3 7 |8 9 6 
3 6 9 |8 4 5 |7 2 1 
2 8 7 |1 6 9 |5 3 4 
------+------+------
5 2 1 |9 7 4 |3 6 8 
4 3 8 |5 2 6 |9 1 7 
7 9 6 |3 1 8 |4 5 2 

